# Training various Matrix Factorization models on ML-100k in PyTorch Lightning Framework

**(Other) Tutorials**
1. MovieLens 1m Recommenders in PyTorch Lightning. [![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1jB5fczYDsF3qOJUXtBMgt-zt_7LyLGG_?usp=sharing)
2. BERT4Rec on ML-25m in PyTorch Lightning. [![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/sparsh-ai/fb375c4a0d8c0c80b025a12c40dc2733/t595874-bert4rec-on-ml-25m-in-pytorch-lightning.ipynb)
3. Implicit Hybrid Movie Recommender using Collie Library. [![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/sparsh-ai/290194a71f907f4544de1ca426a34246/t660394-implicit-hybrid-movie-recommender-using-collie-library.ipynb)
4. Training various Matrix Factorization models on ML-100k in PyTorch Lightning Framework. [![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/recohut/notebooks/blob/main/nbs/P499733_Training_various_Matrix_Factorization_models_on_ML_100k_in_PyTorch_Lightning_Framework.ipynb)

In [ ]:
!pip install -q pytorch_lightning
!pip install -q git+https://github.com/RecoHut-Projects/recohut.git

In [ ]:
from abc import abstractmethod
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable

from tqdm.notebook import tqdm
import sys
import os
from os import path as osp
from pathlib import Path
from collections import OrderedDict
import scipy.sparse as sparse
import collections
import random

import random
import numpy as np
import pandas as pd
from typing import Optional
from scipy.sparse import coo_matrix

import torch
from torch import nn
from torch.nn import Linear
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split

import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.seed import seed_everything

from recohut.utils.common_utils import *
from recohut.datasets import base

import warnings
warnings.filterwarnings('ignore')

## Dataset

In [ ]:
class ML1mDataset(torch.utils.data.Dataset, base.Dataset):
    url = "http://files.grouplens.org/datasets/movielens/ml-1m.zip"

    def __init__(self, args, train=None):
        self.min_rating = args.min_rating
        self.min_uc = args.min_uc
        self.min_sc = args.min_sc
        self.num_negative_samples = args.num_negative_samples
        self.max_number_of_samples_to_consider = 200
        self.is_train = train

        super().__init__(args.data_dir)

        assert self.min_uc >= 2, 'Need at least 2 ratings per user for validation and test'

        self._process()

        if self.is_train is not None:
            self.load()

    @property
    def raw_file_names(self):
        return 'ratings.dat'

    @property
    def processed_file_names(self):
        return ['ml_1m_train.pt', 'ml_1m_test_pos.pt', 'ml_1m_test_neg.pt']

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        move(osp.join(self.raw_dir, 'ml-1m', self.raw_file_names), self.raw_dir)
        rmtree(osp.join(self.raw_dir, 'ml-1m'))
        os.unlink(path)

    def make_implicit(self, df):
        "convert the explicit data to implicit by only keeping interactions with a rating >= min_rating"
        print('Turning into implicit ratings')
        df = df[df['rating'] >= self.min_rating].reset_index(drop=True)
        df['rating'] = 1
        return df

    def filter_triplets(self, df):
        print('Filtering triplets')
        if self.min_sc > 0 or self.min_uc > 0:
            item_sizes = df.groupby('sid').size()
            good_items = item_sizes.index[item_sizes >= self.min_sc]
            user_sizes = df.groupby('uid').size()
            good_users = user_sizes.index[user_sizes >= self.min_uc]
            while len(good_items) < len(item_sizes) or len(good_users) < len(user_sizes):
                if self.min_sc > 0:
                    item_sizes = df.groupby('sid').size()
                    good_items = item_sizes.index[item_sizes >= self.min_sc]
                    df = df[df['sid'].isin(good_items)]

                if self.min_uc > 0:
                    user_sizes = df.groupby('uid').size()
                    good_users = user_sizes.index[user_sizes >= self.min_uc]
                    df = df[df['uid'].isin(good_users)]

                item_sizes = df.groupby('sid').size()
                good_items = item_sizes.index[item_sizes >= self.min_sc]
                user_sizes = df.groupby('uid').size()
                good_users = user_sizes.index[user_sizes >= self.min_uc]
        return df

    def densify_index(self, df):
        print('Densifying index')
        umap = {u: i for i, u in enumerate(set(df['uid']))}
        smap = {s: i for i, s in enumerate(set(df['sid']))}
        df['uid'] = df['uid'].map(umap)
        df['sid'] = df['sid'].map(smap)
        return df, umap, smap

    def load_ratings_df(self):
        df = pd.read_csv(self.raw_paths[0], sep='::', header=None, engine='python')
        df.columns = ['uid', 'sid', 'rating', 'timestamp']
        # drop duplicate user-item pair records, keeping recent ratings only
        df.drop_duplicates(subset=['uid', 'sid'], keep='last', inplace=True)
        return df

    @staticmethod
    def _subset_interactions(mat, idxs):
        idxs = np.array(idxs)

        coo_mat = coo_matrix(
            (mat.data[idxs], (mat.row[idxs], mat.col[idxs])),
            shape=(mat.shape[0], mat.shape[1])
        )

        return coo_mat

    def random_split(self,
                     mat,
                     val_p = 0.0,
                     test_p = 0.2,
                     seed = 42):
        """Randomly split interactions into training, validation, and testing sets."""

        np.random.seed(seed)

        num_interactions = mat.nnz

        shuffle_indices = np.arange(num_interactions)
        np.random.shuffle(shuffle_indices)

        interactions = self._subset_interactions(mat=mat,
                                            idxs=shuffle_indices)

        validate_and_test_p = val_p + test_p
        validate_cutoff = int((1.0 - validate_and_test_p) * num_interactions)
        test_cutoff = int((1.0 - test_p) * num_interactions)

        train_idxs = np.arange(validate_cutoff)
        validate_idxs = np.arange(validate_cutoff, test_cutoff)
        test_idxs = np.arange(test_cutoff, num_interactions)

        train_interactions = self._subset_interactions(mat=mat,
                                                idxs=train_idxs)
        test_interactions = self._subset_interactions(mat=mat,
                                                idxs=test_idxs)

        if val_p > 0:
            validate_interactions = self._subset_interactions(mat=mat,
                                                        idxs=validate_idxs)

            return train_interactions, validate_interactions, test_interactions
        else:
            return train_interactions, test_interactions

    @staticmethod
    def _convert_to_torch_sparse(mat):
        values = mat.data
        indices = np.vstack((mat.row, mat.col))

        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        shape = mat.shape

        return torch.sparse.FloatTensor(i, v, torch.Size(shape))

    def process(self):
        df = self.load_ratings_df()
        if self.min_rating:
            df = self.make_implicit(df)
        df = self.filter_triplets(df)
        df, umap, smap = self.densify_index(df)
        self.num_users = max(df.uid) + 1 # df.uid.nunique()
        self.num_items = max(df.sid) + 1 # df.sid.nunique()
        mat = coo_matrix((np.array(df.rating),
                          (np.array(df.uid), np.array(df.sid))),
                         shape=(self.num_users, self.num_items))
        
        self.positive_items = set(zip(mat.row, mat.col))

        mat_train, mat_test = self.random_split(mat)

        mat_train = self._convert_to_torch_sparse(mat_train)
        torch.save(mat_train, self.processed_paths[0])

        mat_test_pos = self._convert_to_torch_sparse(mat_test)._indices().T 
        _, indices = np.unique(mat_test_pos[:, 0], return_index=True)
        mat_test_pos = mat_test_pos[indices, :]
        torch.save(mat_test_pos, self.processed_paths[1])

        mat_test_neg = self._negative_sample(np.arange(mat_test.shape[0]))
        mat_test_neg = torch.tensor(mat_test_neg, dtype=torch.int)
        torch.save(mat_test_neg, self.processed_paths[2])

        return mat
        
    def todense(self) -> np.matrix:
        """Transforms sparse matrix to np.matrix, 2-d."""
        return self.mat.todense()

    def toarray(self) -> np.array:
        """Transforms sparse matrix to np.array, 2-d."""
        return self.mat.toarray()

    def head(self, n: int = 5) -> np.array:
        """Return the first ``n`` rows of the dense matrix as a np.array, 2-d."""
        n = self._prep_head_tail_n(n=n)
        return self.mat.tocsr()[range(n), :].toarray()

    def tail(self, n: int = 5) -> np.array:
        """Return the last ``n`` rows of the dense matrix as a np.array, 2-d."""
        n = self._prep_head_tail_n(n=n)
        return self.mat.tocsr()[range(-n, 0), :].toarray()

    def _prep_head_tail_n(self, n: int) -> int:
        """Ensure we don't run into an ``IndexError`` when using ``head`` or ``tail`` methods."""
        if n < 0:
            n = self.num_users + n
        if n > self.num_users:
            n = self.num_users
        return n

    def _negative_sample(self, user_id: Union[int, np.array]) -> np.array:
        """Generate negative samples for a ``user_id``."""
        if self.max_number_of_samples_to_consider > 0:
            # if we are here, we are doing true negative sampling
            negative_item_ids_list = list()

            if not isinstance(user_id, collections.abc.Iterable):
                user_id = [user_id]

            for specific_user_id in user_id:
                # generate true negative samples for the ``user_id``
                samples_checked = 0
                temp_negative_item_ids_list = list()

                while len(temp_negative_item_ids_list) < self.num_negative_samples:
                    negative_item_id = random.choice(range(self.num_items))
                    # we have a negative sample, make sure the user has not interacted with it
                    # before, else we resample and try again
                    while (
                        (specific_user_id, negative_item_id) in self.positive_items
                        or negative_item_id in temp_negative_item_ids_list
                    ):
                        if samples_checked >= self.max_number_of_samples_to_consider:
                            num_samples_left_to_generate = (
                                self.num_negative_samples - len(temp_negative_item_ids_list) - 1
                            )
                            temp_negative_item_ids_list += random.choices(
                                range(self.num_items), k=num_samples_left_to_generate
                            )
                            break

                        negative_item_id = random.choice(range(self.num_items))
                        samples_checked += 1

                    temp_negative_item_ids_list.append(negative_item_id)

                negative_item_ids_list += [np.array(temp_negative_item_ids_list)]

            if len(user_id) > 1:
                negative_item_ids_array = np.stack(negative_item_ids_list)
            else:
                negative_item_ids_array = negative_item_ids_list[0]
        else:
            # if we are here, we are doing approximate negative sampling
            if isinstance(user_id, collections.abc.Iterable):
                size = (len(user_id), self.num_negative_samples)
            else:
                size = (self.num_negative_samples,)

            negative_item_ids_array = np.random.randint(
                low=0,
                high=self.num_items,
                size=size,
            )

        return negative_item_ids_array

    def load(self):
        if self.is_train:
            self.train = torch.load(self.processed_paths[0])
            self.train_pos = self.train._indices().T
            self.n_users, self.n_items = self.train.size()

            self.score = torch.sparse.sum(self.train, dim=0).to_dense().repeat((self.n_users, 1))
            self.score[self.train_pos[:, 0], self.train_pos[:, 1]] = 0
        else:
            self.test_pos = torch.load(self.processed_paths[1])
            self.test_neg = torch.load(self.processed_paths[2])
            self.n_users = self.test_pos.shape[0]

            test_items = []
            for u in range(self.n_users):
                items = torch.cat((self.test_pos[u, 1].view(1), self.test_neg[u]))
                test_items.append(items)

            self.test_items = torch.vstack(test_items)
            self.test_labels = torch.zeros(self.test_items.shape)
            self.test_labels[:, 0] += 1


    def __len__(self):
            return self.n_users

    def __train__(self, index):
            return self.train_pos[index], self.score[self.train_pos[index][0]]

    def __test__(self, index):
            return self.test_pos[index], self.test_items[index], self.test_labels[index]

    def __getitem__(self, index):
            if self.is_train:
                return self.__train__(index)
            else:
                return self.__test__(index)

In [ ]:
class ML1mDataModule(LightningDataModule):

    def __init__(self, args) -> None:
        self.args = args
        super().__init__(args.data_dir)

        self.data_dir = args.data_dir if args.data_dir is not None else os.getcwd()
        self.val_split = args.val_split
        self.num_workers = args.num_workers
        self.normalize = args.normalize
        self.batch_size = args.batch_size
        self.seed = args.seed
        self.shuffle = args.shuffle
        self.pin_memory = args.pin_memory
        self.drop_last = args.drop_last

    def prepare_data(self, *args: Any, **kwargs: Any) -> None:
        """Saves files to data_dir."""
        self.data = ML1mDataset(self.args)

    def setup(self, stage: Optional[str] = None) -> None:
        """Creates train, val, and test dataset."""
        if stage == "fit" or stage is None:
            dataset_train = ML1mDataset(self.args, train=True)
            dataset_val = ML1mDataset(self.args, train=True)

            # Split
            self.dataset_train = self._split_dataset(dataset_train)
            self.dataset_val = self._split_dataset(dataset_val, train=False)

        if stage == "test" or stage is None:
            self.dataset_test = ML1mDataset(self.args, train=False)

    def _split_dataset(self, dataset: Dataset, train: bool = True) -> Dataset:
        """Splits the dataset into train and validation set."""
        len_dataset = len(dataset)
        splits = self._get_splits(len_dataset)
        dataset_train, dataset_val = random_split(dataset, splits, generator=torch.Generator().manual_seed(self.seed))

        if train:
            return dataset_train
        return dataset_val

    def _get_splits(self, len_dataset: int) -> List[int]:
        """Computes split lengths for train and validation set."""
        if isinstance(self.val_split, int):
            train_len = len_dataset - self.val_split
            splits = [train_len, self.val_split]
        elif isinstance(self.val_split, float):
            val_len = int(self.val_split * len_dataset)
            train_len = len_dataset - val_len
            splits = [train_len, val_len]
        else:
            raise ValueError(f"Unsupported type {type(self.val_split)}")

        return splits

    def train_dataloader(self, *args: Any, **kwargs: Any) -> DataLoader:
        """The train dataloader."""
        return self._data_loader(self.dataset_train, shuffle=self.shuffle)

    def val_dataloader(self, *args: Any, **kwargs: Any) -> Union[DataLoader, List[DataLoader]]:
        """The val dataloader."""
        return self._data_loader(self.dataset_val)

    def test_dataloader(self, *args: Any, **kwargs: Any) -> Union[DataLoader, List[DataLoader]]:
        """The test dataloader."""
        return self._data_loader(self.dataset_test)

    def _data_loader(self, dataset: Dataset, shuffle: bool = False) -> DataLoader:
        return DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=shuffle,
            num_workers=self.num_workers,
            drop_last=self.drop_last,
            pin_memory=self.pin_memory,
        )

## Model

In [ ]:
def get_ncdg(true, pred):
    match = pred.eq(true).nonzero(as_tuple=True)[1]
    ncdg = torch.log(torch.Tensor([2])).div(torch.log(match + 2))
    ncdg = ncdg.sum().div(pred.shape[0]).item()

    return ncdg


def get_apak(true, pred):
    k = pred.shape[1]
    apak = pred.eq(true).div(torch.arange(k) + 1)
    apak = apak.sum().div(pred.shape[0]).item()

    return apak


def get_hr(true, pred):
    hr = pred.eq(true).sum().div(pred.shape[0]).item()

    return hr


def get_eval_metrics(scores, true, k=10):
    test_items = [torch.LongTensor(list(item_scores.keys())) for item_scores in scores]
    test_scores = [torch.Tensor(list(item_scores.values())) for item_scores in scores]
    topk_indices = [s.topk(k).indices for s in test_scores]
    topk_items = [item[idx] for item, idx in zip(test_items, topk_indices)]
    pred = torch.vstack(topk_items)
    ncdg = get_ncdg(true, pred)
    apak = get_apak(true, pred)
    hr = get_hr(true, pred)

    return ncdg, apak, hr

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, n_neg=4, k=10):
        super().__init__()
        self.n_neg = n_neg
        self.k = k

    def forward(self, users, items):
        raise NotImplementedError

    def training_step(self, batch, batch_idx):
        pos, score = batch
        users, pos_items = pos[:, 0], pos[:, 1]

        neg_items = torch.multinomial(score, self.n_neg)
        items = torch.cat((pos_items.view(-1, 1), neg_items), dim=1)

        labels = torch.zeros(items.shape)
        labels[:, 0] += 1
        users = users.view(-1, 1).repeat(1, items.shape[1])

        users = users.view(-1, 1).squeeze()
        items = items.view(-1, 1).squeeze()
        labels = labels.view(-1, 1).squeeze()

        logits = self(users, items)
        loss = self.loss_fn(logits, labels)

        return {
            "loss": loss,
            "logits": logits.detach(),
        }

    def training_epoch_end(self, outputs):
        # This function recevies as parameters the output from "training_step()"
        # Outputs is a list which contains a dictionary like:
        # [{'pred':x,'target':x,'loss':x}, {'pred':x,'target':x,'loss':x}, ...]
        pass

    def test_step(self, batch, batch_idx):
        pos, items, labels = batch
        n_items = items.shape[1]
        users = pos[:, 0].view(-1, 1).repeat(1, n_items)

        users = users.view(-1, 1).squeeze()
        items = items.view(-1, 1).squeeze()
        labels = labels.view(-1, 1).squeeze()

        logits = self(users, items)
        loss = self.loss_fn(logits, labels)

        items = items.view(-1, n_items)
        logits = logits.view(-1, n_items)
        item_true = pos[:, 1].view(-1, 1)
        item_scores = [dict(zip(item.tolist(), score.tolist())) for item, score in zip(items, logits)]
        ncdg, apak, hr = get_eval_metrics(item_scores, item_true, self.k)
        metrics = {
            'loss': loss.item(),
            'ncdg': ncdg,
            'apak': apak,
            'hr': hr,
        }
        self.log("Val Metrics", metrics, prog_bar=True)

        return {
            "loss": loss.item(),
            "logits": logits,
        }

    def test_epoch_end(self, outputs):
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.005)
        return optimizer

    def loss_fn(self, logits, labels):
        return nn.BCEWithLogitsLoss()(logits, labels)

In [ ]:
class MF(Model):
    """A matrix factorization model trained using SGD and negative sampling."""

    def __init__(self, n_users, n_items, embedding_dim):
        super().__init__()
        self.user_embedding = nn.Embedding(
            num_embeddings=n_users, embedding_dim=embedding_dim
        )
        self.item_embedding = nn.Embedding(
            num_embeddings=n_items, embedding_dim=embedding_dim
        )
        self.user_bias = nn.Parameter(torch.zeros((n_users)))
        self.item_bias = nn.Parameter(torch.zeros((n_items)))
        self.bias = nn.Parameter(torch.Tensor([0]))

    def forward(self, users, items):
        return (
                self.bias +
                self.user_bias[users] +
                self.item_bias[items] +
                (self.user_embedding(users).mul(self.item_embedding(items))).sum(dim=-1)
        )

In [ ]:
class GMF(Model):
    def __init__(self, n_users, n_items, embedding_dim):
        super().__init__()

        self.user_embedding = nn.Embedding(
            num_embeddings=n_users, embedding_dim=embedding_dim
        )
        self.item_embedding = nn.Embedding(
            num_embeddings=n_items, embedding_dim=embedding_dim
        )
        self.fc = nn.Linear(embedding_dim, 1)
        
        # not using sigmoid layer because loss is BCEWithLogits in PairModel
        # self.logistic = nn.Sigmoid()

    def forward(self, users, items):
        user_embeddings = self.user_embedding(users)
        item_embeddings = self.item_embedding(items)
        embeddings = user_embeddings.mul(item_embeddings)
        output = self.fc(embeddings)

        # not using sigmoid layer because loss is BCEWithLogits in PairModel
        # rating = self.logistic(output)

        return output.squeeze()

In [ ]:
class MLP(Model):
    def __init__(self, n_users, n_items, embedding_dim, dropout=0.1):
        super().__init__()

        self.user_embedding = nn.Embedding(
            num_embeddings=n_users, embedding_dim=embedding_dim
        )
        self.item_embedding = nn.Embedding(
            num_embeddings=n_items, embedding_dim=embedding_dim
        )
        self.fc1 = nn.Linear(embedding_dim * 2, embedding_dim)
        self.fc2 = nn.Linear(embedding_dim, int(embedding_dim / 2))
        self.fc3 = nn.Linear(int(embedding_dim / 2), 1)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, users, items):
        user_embeddings = self.user_embedding(users)
        item_embeddings = self.item_embedding(items)
        embeddings = torch.cat([user_embeddings, item_embeddings], axis=1)
        output = nn.ReLU()(self.fc1(embeddings))
        output = self.dropout(output)
        output = nn.ReLU()(self.fc2(output))
        output = self.dropout(output)
        output = self.fc3(output)

        return output.squeeze()

In [ ]:
class NeuMF(Model):
    def __init__(self, n_users, n_items, embedding_dim, dropout=0.1):
        super().__init__()

        self.user_embedding = nn.Embedding(
            num_embeddings=n_users, embedding_dim=embedding_dim
        )
        self.item_embedding = nn.Embedding(
            num_embeddings=n_items, embedding_dim=embedding_dim
        )

        self.user_embedding_gmf = nn.Embedding(
            num_embeddings=n_users, embedding_dim=embedding_dim
        )
        self.item_embedding_gmf = nn.Embedding(
            num_embeddings=n_items, embedding_dim=embedding_dim
        )

        self.gmf = nn.Linear(embedding_dim, int(embedding_dim / 2))

        self.fc1 = nn.Linear(embedding_dim * 2, embedding_dim)
        self.fc2 = nn.Linear(embedding_dim, embedding_dim)
        self.fc3 = nn.Linear(embedding_dim, int(embedding_dim / 2))

        self.fc_final = nn.Linear(embedding_dim, 1)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, users, items):
        user_embeddings = self.user_embedding(users)
        item_embeddings = self.item_embedding(items)
        embeddings = torch.cat([user_embeddings, item_embeddings], dim=1)

        user_embeddings_gmf = self.user_embedding_gmf(users)
        item_embeddings_gmf = self.item_embedding_gmf(items)
        embeddings_gmf = user_embeddings_gmf.mul(item_embeddings_gmf)

        output_gmf = self.gmf(embeddings_gmf)
        output = nn.ReLU()(self.fc1(embeddings))
        output = self.dropout(output)
        output = nn.ReLU()(self.fc2(output))
        output = self.dropout(output)
        output = self.fc3(output)

        output = torch.cat([output, output_gmf], dim=1)
        output = self.fc_final(output)

        return output.squeeze()

## Trainer

In [ ]:
class Args:
    data_dir = '/content/data' # Where to save/load the data
    min_rating = 4
    num_negative_samples = 99
    min_uc = 5
    min_sc = 5

    log_dir = '/content/logs'
    model_dir = '/content/models'

    val_split = 0.2 # Percent (float) or number (int) of samples to use for the validation split
    num_workers = 2 # How many workers to use for loading data
    normalize = False # If true applies rating normalize
    batch_size = 32 # How many samples per batch to load
    seed = 42 # Random seed to be used for train/val/test splits
    shuffle = True # If true shuffles the train data every epoch
    pin_memory = True # If true, the data loader will copy Tensors into CUDA pinned memory before returning them
    drop_last = False # If true drops the last incomplete batch

    embedding_dim = 20
    max_epochs = 5

args = Args()

In [ ]:
ds = ML1mDataModule(args)

logger = TensorBoardLogger(
    save_dir=args.log_dir,
)

checkpoint_callback = ModelCheckpoint(
    monitor="valid_loss",
    mode="min",
    dirpath=args.model_dir,
    filename="recommender",
)

def pl_trainer(model, datamodule):

    trainer = pl.Trainer(
    max_epochs=args.max_epochs,
    logger=logger,
    check_val_every_n_epoch=10,
    callbacks=[checkpoint_callback],
    # enable_checkpointing=False,
    # num_sanity_val_steps=0,
    # gradient_clip_val=1,
    # gradient_clip_algorithm="norm",
    gpus=None
    )

    trainer.fit(model, datamodule=datamodule)
    test_result = trainer.test(model, datamodule=datamodule)
    return test_result

In [ ]:
ds.prepare_data()

Processing...


Turning into implicit ratings
Filtering triplets
Densifying index


Done!


In [ ]:
model = MF(n_items=ds.data.num_items, n_users=ds.data.num_users, embedding_dim=args.embedding_dim)

pl_trainer(model, ds)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 120 K 
1 | item_embedding | Embedding | 62.5 K
---------------------------------------------
192 K     Trainable params
0         Non-trainable params
192 K     Total params
0.769     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Val Metrics': {'apak': tensor(0.0296),
                 'hr': tensor(0.1093),
                 'loss': tensor(1.0958),
                 'ncdg': tensor(0.0477)}}
--------------------------------------------------------------------------------


[{'Val Metrics': {'apak': tensor(0.0296),
   'hr': tensor(0.1093),
   'loss': tensor(1.0958),
   'ncdg': tensor(0.0477)}}]

In [ ]:
model = NeuMF(n_items=ds.data.num_items, n_users=ds.data.num_users, embedding_dim=args.embedding_dim)

pl_trainer(model, ds)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name               | Type      | Params
-------------------------------------------------
0 | user_embedding     | Embedding | 120 K 
1 | item_embedding     | Embedding | 62.5 K
2 | user_embedding_gmf | Embedding | 120 K 
3 | item_embedding_gmf | Embedding | 62.5 K
4 | gmf                | Linear    | 210   
5 | fc1                | Linear    | 820   
6 | fc2                | Linear    | 420   
7 | fc3                | Linear    | 210   
8 | fc_final           | Linear    | 21    
9 | dropout            | Dropout   | 0     
-------------------------------------------------
368 K     Trainable params
0         Non-trainable params
368 K     Total params
1.472     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Val Metrics': {'apak': tensor(0.0675),
                 'hr': tensor(0.2177),
                 'loss': tensor(0.2396),
                 'ncdg': tensor(0.1024)}}
--------------------------------------------------------------------------------


[{'Val Metrics': {'apak': tensor(0.0675),
   'hr': tensor(0.2177),
   'loss': tensor(0.2396),
   'ncdg': tensor(0.1024)}}]

In [ ]:
model = GMF(n_items=ds.data.num_items, n_users=ds.data.num_users, embedding_dim=args.embedding_dim)

pl_trainer(model, ds)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 120 K 
1 | item_embedding | Embedding | 62.5 K
2 | fc             | Linear    | 21    
---------------------------------------------
183 K     Trainable params
0         Non-trainable params
183 K     Total params
0.733     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Val Metrics': {'apak': tensor(0.0384),
                 'hr': tensor(0.1208),
                 'loss': tensor(0.2579),
                 'ncdg': tensor(0.0574)}}
--------------------------------------------------------------------------------


[{'Val Metrics': {'apak': tensor(0.0384),
   'hr': tensor(0.1208),
   'loss': tensor(0.2579),
   'ncdg': tensor(0.0574)}}]

In [ ]:
model = MLP(n_items=ds.data.num_items, n_users=ds.data.num_users, embedding_dim=args.embedding_dim)

pl_trainer(model, ds)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 120 K 
1 | item_embedding | Embedding | 62.5 K
2 | fc1            | Linear    | 820   
3 | fc2            | Linear    | 210   
4 | fc3            | Linear    | 11    
5 | dropout        | Dropout   | 0     
---------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.737     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Val Metrics': {'apak': tensor(0.0958),
                 'hr': tensor(0.3172),
                 'loss': tensor(0.2204),
                 'ncdg': tensor(0.1467)}}
--------------------------------------------------------------------------------


[{'Val Metrics': {'apak': tensor(0.0958),
   'hr': tensor(0.3172),
   'loss': tensor(0.2204),
   'ncdg': tensor(0.1467)}}]

---

In [ ]:
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2022-01-09 14:28:48

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

pytorch_lightning: 1.5.8
recohut          : 0.0.10
IPython          : 5.5.0
sys              : 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
scipy            : 1.4.1
torch            : 1.10.0+cu111
numpy            : 1.19.5
pandas           : 1.1.5



---

**END**